### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [4]:
import csv
import pandas as pd
import numpy as np
import json

In [5]:
import os
import pandas as pd

# 读取标签文件
label = pd.read_csv('labelA/回应情况-表格 1.csv')

In [13]:
label.head()


,切片ID,回应情况,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,强,中,弱,没有回应（忽视）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15YS_20230317_01/VCAM_0000,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15YS_20230317_01/VCAM_0000_1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15YS_20230317_01/VCAM_0000_2,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15YS_20230317_01/VCAM_0000_3,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
label.iloc[1,0][-4:]

'0000'

### 导入face特征

In [ ]:
# set confidence threshold
confidence_threshold = 0.5

# 导入face特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0
for i in range(1,len(label)):
    directory = label.iloc[i, 0]

    label_value = label.iloc[i, 1:5] # 取1-4位的标签
    feature_file = os.path.join('siblings_face', directory + '.csv')
    try:
        feature_data = pd.read_csv(feature_file)

        # 丢弃置信度小于0.8的数据
        feature_data = feature_data[feature_data['confidence'] >= confidence_threshold]
        
        # 每一frame加入
        features.append(feature_data.values) 


        # 重复frame标签
        for j in range(feature_data.shape[0]):
            labels.append(label_value.astype("int"))
        
        Dcount += 1
    except:
        print(f"error: {feature_file} not exist")
        Mcount +=1
        continue
    

print(f"Import {Dcount} files, missing {Mcount} files")

# 将特征和标签转换为numpy数组
# X = np.concatenate(features, axis=0)
X = np.array(features)
y = np.array(labels)

print(f"X:{len(X)}, y:{len(y)}")


In [ ]:
0.8 # 16000
0.7 # 17393
0.6 # 18295
0.5 # 19325

### 导入pose特征

In [ ]:
import json

# 导入特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0

# for a single video
for i in range(1,len(label)):
    directory = label.iloc[i, 0] 
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    # label
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    pose_json_data = []

    # for a single frame
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # 漏检测 只有1人的情况
            if len(data['people']) != 2:
                row0 = data['people'][0]['pose_keypoints_2d']
                # 插入face_id
                row0.insert(0,0)
                 # 插入frame
                frame_id = int(file.split('_')[-2])//5 +1 # 恢复frame id
                row0.insert(0,frame_id)
                pose_json_data.append(row0)

                continue
            
            row0 = data['people'][0]['pose_keypoints_2d']
            row1 = data['people'][1]['pose_keypoints_2d']


            # face_id对应 x较小 标记为1，x较大 标记为0 
            if(row0[0]<row1[0]):
                row0,row1 = row1,row0 # swap

            # 插入face_id
            row0.insert(0,0)
            row1.insert(0,1)

            # 插入frame
            frame_id = int(file.split('_')[-2])//5 # 恢复frame id
            row0.insert(0,frame_id)
            row1.insert(0,frame_id)


            # 重复frame标签
            # labels.append(label_value.astype("int"))
            pose_json_data.append(row0)
            pose_json_data.append(row1)

        except:
            print(f"pose error: {file} not exist")
            continue

    features.append(np.array(pose_json_data))
    

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"Import {Dcount} files, missing {Mcount} files")
print(f"X:{len(X)}, y:{len(y)}")

### 合并提取

In [14]:
# set openface confidence threshold
confidence_threshold = 0.5

# 导入特征并对齐标签
features = []
labels = []
# 记录原本有多少条记录
face_count = 0
pose_count = 0


for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct Openpose directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    

    # the correct Openface csv file
    feature_file = os.path.join('siblings_face', directory + '.csv')

    # the label data for this video
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    # Load face data of a video
    try:
        face_csv_data = pd.read_csv(feature_file)

        # 丢弃置信度小于0.5的数据
        face_csv_data = face_csv_data[face_csv_data['confidence'] >= confidence_threshold]

        # 找到frame中有多于2个不同face_id的帧
        counts = face_csv_data.groupby('frame')['face_id'].nunique()
        frames_to_drop = counts[counts > 2].index.tolist()

        # 对于每个需要处理的帧，删除置信度最低的行，直到只剩下2个face_id
        for frame in frames_to_drop:
            frame_df = face_csv_data[face_csv_data['frame'] == frame]
            while frame_df['face_id'].nunique() > 2:
                min_confidence = frame_df['confidence'].min()
                rows_to_drop = frame_df[(frame_df['confidence'] == min_confidence)]['face_id'].tolist()
                frame_df = frame_df[~frame_df['face_id'].isin(rows_to_drop)]
            face_csv_data = face_csv_data[face_csv_data['frame'] != frame]
            face_csv_data = pd.concat([face_csv_data, frame_df], ignore_index=True)

        # 每一frame的 face feature
        face_count += face_csv_data.shape[0]
        features_face = face_csv_data.values 
    except:
        print(f"face error: {feature_file} not exist")
        continue
    
    pose_json_data = [] # save all frame data
    # Load pose data of a video frame
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # 漏检测 只有1人的情况
            if len(data['people']) != 2:
                row0 = data['people'][0]['pose_keypoints_2d']
                # 插入face_id
                row0.insert(0,0)
                 # 插入frame
                frame_id = int(file.split('_')[-2])//5 # 恢复frame id
                row0.insert(0,frame_id)
                pose_json_data.append(row0)

                continue
            
            row0 = data['people'][0]['pose_keypoints_2d']
            row1 = data['people'][1]['pose_keypoints_2d']


            # face_id对应 x较小 标记为1，x较大 标记为0 
            if(row0[0]<row1[0]):
                row0,row1 = row1,row0 # swap

            # 插入face_id
            row0.insert(0,0)
            row1.insert(0,1)

            # 插入frame
            frame_id = int(file.split('_')[-2])//5 + 1 # 恢复frame id 
            row0.insert(0,frame_id)
            row1.insert(0,frame_id)
            
            pose_json_data.append(row0)
            pose_json_data.append(row1)

        except:
            print(f"pose error: {file} not exist")
            continue

    pose_count += np.array(pose_json_data).shape[0]
    features_pose = np.array(pose_json_data)

    # check len
    # print(f"face:{len(face_csv_data)}, pose:{len(pose_json_data)}")
    
    # 将两个数组合并为一个数组
    for face in features_face:
        for pose in features_pose:
            if np.array_equal(face[:2], pose[:2]):
                # 合并face和pose
                features.append(np.concatenate((face[2:], pose[2:])))
                # 重复labels
                labels.append(label_value.astype("int"))
                break
            else:
                continue
            break

# 将特征和标签转换为numpy数组
X = np.array(features)
y = np.array(labels)

# 总共遍历的face特征和pose特征数
print(f"Total face feat {face_count}, pose feat {pose_count}")
# 读入的数量
print(f"Read in X:{len(X)}, y:{len(y)}")
# drop
print(f"Drop {face_count - len(X)} face features, {pose_count - len(X)} pose features")


face error: siblings_face/15YS_20230317_01/VCAM_0000_3.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0000_37.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0001_29.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0001_30.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0001_37.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0002_53.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0002_76.csv not exist
face error: siblings_face/15YS_20230317_01/VCAM_0003_89.csv not exist
face error: siblings_face/16YS_20230317_01/VCAM_0004_126.csv not exist
pose error: VCAM_0005_164_000000000166_keypoints.json not exist
face error: siblings_face/16YS_20230317_01/VCAM_0006_18.csv not exist
face error: siblings_face/16YS_20230317_01/VCAM_0006_81.csv not exist
face error: siblings_face/16YS_20230317_01/VCAM_0006_103.csv not exist
pose error: VCAM_0006_142_000000000190_keypoints.json not exist
Total face feat 19255, pose fea

维度

In [23]:
X = np.array(features)
print(f"X:{len(X[0])}")

X:789


### Train

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [32]:

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# 使用随机森林预测一个frame
rfc = RandomForestClassifier(n_estimators=25, max_depth=10)
rfc.fit(X_train, y_train)
y_predict = rfc.predict(X_test)

# make predictions on the test set
y_predict_proba = np.array(rfc.predict_proba(X_test))
# y_predict = np.zeros_like(y_predict_proba)
# y_predict[np.arange(len(y_predict_proba)), y_predict_proba.argmax(1)] = 1

print(f"acc:{rfc.score(X_test, y_test)}")
print(classification_report(y_test, y_predict, target_names=["强","中","弱","没有回应（忽视）"]))

acc:0.42942008486562944
              precision    recall  f1-score   support

           强       0.96      0.45      0.62      1074
           中       1.00      0.15      0.26       559
           弱       0.97      0.43      0.59      1057
    没有回应（忽视）       0.94      0.58      0.72       845

   micro avg       0.96      0.43      0.59      3535
   macro avg       0.97      0.40      0.55      3535
weighted avg       0.96      0.43      0.58      3535
 samples avg       0.43      0.43      0.43      3535



/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# 统计非[0, 0, 0, 0]的行数
nonzero_rows = np.count_nonzero(np.any(y_predict!= [0, 0, 0, 0], axis=1))

# 打印结果
print(nonzero_rows)


1587


In [36]:
print(y_predict_proba)
print(y_predict_proba.shape)

[[[0.69842307 0.30157693]
  [0.57797997 0.42202003]
  [0.82791906 0.17208094]
  ...
  [0.54693533 0.45306467]
  [0.79626327 0.20373673]
  [0.79293209 0.20706791]]

 [[0.88437842 0.11562158]
  [0.819527   0.180473  ]
  [0.8441718  0.1558282 ]
  ...
  [0.85194001 0.14805999]
  [0.89552325 0.10447675]
  [0.67095994 0.32904006]]

 [[0.77759191 0.22240809]
  [0.7363147  0.2636853 ]
  [0.6326532  0.3673468 ]
  ...
  [0.7449863  0.2550137 ]
  [0.58057047 0.41942953]
  [0.60545355 0.39454645]]

 [[0.6396066  0.3603934 ]
  [0.86617833 0.13382167]
  [0.69525595 0.30474405]
  ...
  [0.85613837 0.14386163]
  [0.72764302 0.27235698]
  [0.93065443 0.06934557]]]
(4, 3535, 2)
